In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow import keras
%pip install git+https://github.com/keras-team/keras-preprocessing.git
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
import os, warnings
import matplotlib.pyplot as plt
from matplotlib import gridspec
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import gc

In [ ]:
import pandas as pd

img_dir = '/content/drive/My Drive/CelebA/Img/img_align_celeba_32500'
# Load labels.csv
labels_df = pd.read_csv('/content/drive/My Drive/CelebA/Anno/labels_even.csv', header=0)

# Load list_bbox_celeba.txt
bbox_df = pd.read_csv('/content/drive/My Drive/CelebA/Anno/list_bbox_celeba.txt', delim_whitespace=True, header=1, dtype=object)

# rename the first column of bbox_df to 'Filename'
bbox_df.rename(columns={"image_id": "Filename"}, inplace=True)

# Convert 'x_1', 'y_1', 'width' and 'height' columns of bbox_df to numeric
bbox_df['x_1'] = pd.to_numeric(bbox_df['x_1'])
bbox_df['y_1'] = pd.to_numeric(bbox_df['y_1'])
bbox_df['width'] = pd.to_numeric(bbox_df['width'])
bbox_df['height'] = pd.to_numeric(bbox_df['height'])

# Merge labels_df and bbox_df
merged_df = pd.merge(labels_df, bbox_df, on='Filename')

In [ ]:
merged_df['x_normalized'] = merged_df['x_1'] / merged_df['width']
merged_df['y_normalized'] = merged_df['y_1'] / merged_df['height']
merged_df['width_normalized'] = merged_df['width'] / merged_df['width']
merged_df['height_normalized'] = merged_df['height'] / merged_df['height']

# remove columns 'x_1', 'y_1', 'width' and 'height'
merged_df.drop(['x_1', 'y_1', 'width', 'height'], axis=1, inplace=True)

In [ ]:
train, valid = train_test_split(merged_df, test_size = 0.3)
valid, test = train_test_split(valid, test_size=0.2)
train_features = train[["High_Cheekbones", "Mouth_Slightly_Open","Smiling"]]
train,valid,test

In [ ]:
# Define list of class names (subset of the classes listed in `list_attr_celeba.txt`)
class_names = ["High_Cheekbones", "Mouth_Slightly_Open", "Smiling"]

# Reproducability
def set_seed(seed=31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    #os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()

def custom_augmentation(np_tensor):

  def random_contrast(np_tensor):
    return tf.image.random_contrast(np_tensor, 0.5, 2)

  augmnted_tensor = random_contrast(np_tensor)
  return np.array(augmnted_tensor)

# Load training and validation sets
datagen=ImageDataGenerator(rescale=1./255.,
                          rotation_range=45,
                          horizontal_flip=True,
                          vertical_flip=True,
                          preprocessing_function=custom_augmentation)
valid_datagen=ImageDataGenerator(rescale=1./255.,
                                rotation_range=45,
                                horizontal_flip=True,
                                vertical_flip=True,
                                preprocessing_function=custom_augmentation)
test_datagen=ImageDataGenerator(rescale=1./255.)

train_generator = datagen.flow_from_dataframe(
    dataframe=train,
    directory=img_dir,
    x_col='Filename',
    y_col=class_names,
    save_format='jpg',
    batch_size=25,
    seed=42,
    shuffle=False,
    class_mode="raw",
    target_size=(178, 218)
)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=valid,
    directory=img_dir,
    x_col='Filename',
    y_col=class_names,
    save_format='jpg',
    batch_size=25,
    seed=42,
    shuffle=True,
    class_mode="raw",
    target_size=(178,218)
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test,
    directory=img_dir,
    x_col='Filename',
    batch_size=15,
    seed=42,
    shuffle=False,
    class_mode=None,
    target_size=(128, 128)
)

# Data Pipeline
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, 128, 128, 3], [None, len(class_names)])
).map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE)

ds_valid = tf.data.Dataset.from_generator(
    lambda: valid_generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, 128, 128, 3], [None, len(class_names)])
).map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE)

'''ds_test = tf.data.Dataset.from_generator(
    lambda: test_generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, 128, 128, 3], [None, len(class_names)])
).map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE)
'''

In [ ]:
import tensorflow.keras.backend as K
K.clear_session()

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Precision, Recall, AUC

# Load pre-trained VGG16 model without top layers
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(218, 178, 3)
)

num_classes = len(class_names)

# Define input layer (adjust the shape based on your input images)
input_tensor = Input(shape=(218, 178, 3), name='input_01')

# Pass input through VGG16 base model
x = base_model(input_tensor)

# Add GlobalAveragePooling2D layer to reduce spatial dimensions
x = GlobalAveragePooling2D()(x)

# Add output layer for class predictions
output_1 = Dense(1, activation='softmax', name='High_Cheekbones')(x)
Dense(1, activation='sigmoid', name='Mouth_Slightly_Open')(x)

# Create the model with a specific name
model = Model(inputs=input_tensor, outputs=class_output, name='VGG16_transfer')

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)

# Print model summary
model.summary()

In [ ]:
inputs = tf.keras.layers.Input(shape=(128, 128, 3))
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.50)(x)
output1 = tf.keras.layers.Dense(1, activation='sigmoid', name='Arched_Eyebrows')(x)
output2 = tf.keras.layers.Dense(1, activation='sigmoid', name='Bags_Under_Eyes')(x)
output3 = tf.keras.layers.Dense(1, activation='sigmoid', name='Bangs')(x)
output4 = tf.keras.layers.Dense(1, activation='sigmoid', name='Black_Hair')(x)
output5 = tf.keras.layers.Dense(1, activation='sigmoid', name='Blond_Hair')(x)
output6 = tf.keras.layers.Dense(1, activation='sigmoid', name='Brown_Hair')(x)
output7 = tf.keras.layers.Dense(1, activation='sigmoid', name='Eyeglasses')(x)
output8 = tf.keras.layers.Dense(1, activation='sigmoid', name='Gray_Hair')(x)
output9 = tf.keras.layers.Dense(1, activation='sigmoid', name='Heavy_Makeup')(x)
output10 = tf.keras.layers.Dense(1, activation='sigmoid', name='High_Cheekbones')(x)
output11 = tf.keras.layers.Dense(1, activation='sigmoid', name='Mouth_Slightly_Open')(x)
output12 = tf.keras.layers.Dense(1, activation='sigmoid', name='Mustache')(x)
output13 = tf.keras.layers.Dense(1, activation='sigmoid', name='Narrow_Eyes')(x)
output14 = tf.keras.layers.Dense(1, activation='sigmoid', name='Rosy_Cheeks')(x)
output15 = tf.keras.layers.Dense(1, activation='sigmoid', name='Smiling')(x)
output16 = tf.keras.layers.Dense(1, activation='sigmoid', name='Straight_Hair')(x)
output17 = tf.keras.layers.Dense(1, activation='sigmoid', name='Wavy_Hair')(x)
output18 = tf.keras.layers.Dense(1, activation='sigmoid', name='Wearing_Earrings')(x)
output19 = tf.keras.layers.Dense(1, activation='sigmoid', name='Wearing_Hat')(x)
output20 = tf.keras.layers.Dense(1, activation='sigmoid', name='Wearing_Lipstick')(x)
output21 = tf.keras.layers.Dense(1, activation='sigmoid', name='Wearing_Necklace')(x)

model = tf.keras.models.Model(inputs=inputs, outputs=[
                                                      output1,
                                                      output2,
                                                      output3,
                                                      output4,
                                                      output5,
                                                      output6,
                                                      output7,
                                                      output8,
                                                      output9,
                                                      output10,
                                                      output11,
                                                      output12,
                                                      output13,
                                                      output14,
                                                      output15,
                                                      output16,
                                                      output17,
                                                      output18,
                                                      output19,
                                                      output20,
                                                      output21
                                                      ])


In [ ]:
model.compile(optimizer='adam',
              loss={'Arched_Eyebrows': 'binary_crossentropy',
              'Bags_Under_Eyes': 'binary_crossentropy',
              'Bangs': 'binary_crossentropy',
              'Black_Hair': 'binary_crossentropy',
              'Blond_Hair': 'binary_crossentropy',
              'Brown_Hair': 'binary_crossentropy',
              'Eyeglasses': 'binary_crossentropy',
              'Gray_Hair': 'binary_crossentropy',
              'Heavy_Makeup': 'binary_crossentropy',
              'High_Cheekbones': 'binary_crossentropy',
              'Mouth_Slightly_Open': 'binary_crossentropy',
              'Mustache': 'binary_crossentropy',
              'Narrow_Eyes': 'binary_crossentropy',
              'Rosy_Cheeks': 'binary_crossentropy',
              'Smiling': 'binary_crossentropy',
              'Straight_Hair': 'binary_crossentropy',
              'Wavy_Hair': 'binary_crossentropy',
              'Wearing_Earrings': 'binary_crossentropy',
              'Wearing_Hat': 'binary_crossentropy',
              'Wearing_Lipstick': 'binary_crossentropy',
              'Wearing_Necklace': 'binary_crossentropy',
              },


              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

gc.enable()

checkpoint = ModelCheckpoint(
    check_save_dir,
    save_weights_only=True,
    save_best_only=True,
    monitor='loss',
    mode='min',
    verbose=1
    )

# define early stopping callback
early_stopping = EarlyStopping(
    monitor='loss', # metric to monitor for early stopping
    patience=3, # number of epochs to wait before stopping
    restore_best_weights=True, # restore the best model weights found during training
)

In [ ]:
def generator_wrapper(dataset):
    for batch_x, batch_y in dataset:
        yield (batch_x, [batch_y[:, i] for i in range(21)])

ds_train_wrapped = generator_wrapper(ds_train)
ds_valid_wrapped = generator_wrapper(ds_valid)

In [ ]:
model.load_weights('./Checkpoints/')

Full Run: 910, 312 <br>
1/5 Run: 182, 78 (1/4)

In [ ]:
history = model.fit(
                    ds_train_wrapped,
                    epochs=2,
                    validation_data=ds_valid_wrapped,
                    batch_size=25,
                    steps_per_epoch=910,
                    validation_steps=312,
                    verbose=1,
                    callbacks=[checkpoint, early_stopping])

In [ ]:
history.history

In [ ]:
gc.collect()

In [ ]:
%cd drive/My Drive/CelebA

In [ ]:
%pwd

In [ ]:
tf.keras.saving.save_model(
    model, './Saved_models/Set_B/model_03', overwrite=False, save_format='tf',
)

In [ ]:
reconstructed_model = keras.models.load_model("./Saved_models/Set_B/model_15")

In [ ]:
reconstructed_model.summary()

In [ ]:
reconstructed_model.load_weights('./Checkpoints/')

In [ ]:
history = reconstructed_model.fit(
                    ds_train_wrapped,
                    epochs=1,
                    validation_data=ds_valid_wrapped,
                    batch_size=25,
                    steps_per_epoch=910,
                    validation_steps=312,
                    verbose=1,
                    callbacks=[checkpoint, early_stopping])

In [ ]:
tf.keras.saving.save_model(
    reconstructed_model, './Saved_models/Set_B/model_15', overwrite=False, save_format='tf',
)

In [ ]:
test_generator.reset()
# Get the predicted probabilities for each class
predictions = reconstructed_model.predict(test_generator, steps=1)

In [ ]:
from PIL import Image
from textwrap import wrap
imagePreds = []

#print(predictions[20][14][0])
for i in range(21):
  classPreds = []
  for j in range(15):
    classPreds.append(predictions[i][j][0])
  imagePreds.append(classPreds)

class_names = ["Arched_Eyebrows", "Bags_Under_Eyes", "Bangs", "Black_Hair", "Blond_Hair", "Brown_Hair", "Eyeglasses", "Gray_Hair", "Heavy_Makeup", "High_Cheekbones", "Mouth_Slightly_Open", "Mustache", "Narrow_Eyes", "Rosy_Cheeks", "Smiling", "Straight_Hair", "Wavy_Hair", "Wearing_Earrings", "Wearing_Hat", "Wearing_Lipstick", "Wearing_Necklace"]
filenames = test['filenames'][:15]
#print(imagePreds)

for i in range(15):
  image = []
  for j in range(21):
    image.append(imagePreds[j][i])
  #print(image)
  sorted_indices = np.argsort(image)
  #print(sorted_indices)
  top_three_indices = sorted_indices[::-1][:3]
  #print(top_three_indices[0])
  top_three_arr = [image[x] for x in top_three_indices]
  #print(top_three_arr)
  xLabel = f"{class_names[sorted_indices[20]]}, {'%.2f' % (top_three_arr[0]*100)}%; {class_names[sorted_indices[19]]}, {'%.2f' % (top_three_arr[1]*100)}%; {class_names[sorted_indices[18]]}, {'%.2f' % (top_three_arr[2]*100)}%"
  #xLabel = ['\n'.join(wrap(l, 20)) for l in xLabel]
  # Plot the results
  plt.figure(figsize=(6, 12))
  #plt.subplot(16, 14, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(Image.open(os.path.join(img_dir, filenames.iloc[i])))
  plt.xlabel(xLabel)
  plt.tight_layout()
  plt.show()


In [ ]:
# set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=16, titlepad=10)
plt.rc('image', cmap='magma')
warnings.filterwarnings("ignore")

# convert history.history object to dataframe
history_frame = pd.DataFrame(history.history)


In [ ]:
print(history_frame)
history_frame.to_csv('/content/drive/My Drive/CelebA/Saved_models/Set_B/metrics/modelB_15.csv', index=False)

In [ ]:

history_frame.loc[:, [
                  'val_Arched_Eyebrows_accuracy',
                  'val_Bags_Under_Eyes_accuracy',
                  'val_Bangs_accuracy',
                  'val_Black_Hair_accuracy',
                  'val_Blond_Hair_accuracy',
                     ]].plot()
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
history_frame.loc[:, [
                  'val_Brown_Hair_accuracy',
                  'val_Eyeglasses_accuracy',
                  'val_Gray_Hair_accuracy',
                  'val_Heavy_Makeup_accuracy',
                  'val_High_Cheekbones_accuracy',
                     ]].plot()
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
history_frame.loc[:, [
                  'val_Mouth_Slightly_Open_accuracy',
                  'val_Mustache_accuracy',
                  'val_Narrow_Eyes_accuracy',
                  'val_Rosy_Cheeks_accuracy',
                  'val_Smiling_accuracy',
                     ]].plot()
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
history_frame.loc[:, [
                  'val_Straight_Hair_accuracy',
                  'val_Wavy_Hair_accuracy',
                  'val_Wearing_Earrings_accuracy',
                  'val_Wearing_Hat_accuracy',
                  'val_Wearing_Lipstick_accuracy',
                  'val_Wearing_Necklace_accuracy',
                  ]].plot()
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

In [ ]:

history_frame.loc[:, [
                  'val_Arched_Eyebrows_loss',
                  'val_Bags_Under_Eyes_loss',
                  'val_Bangs_loss',
                  'val_Black_Hair_loss',
                  'val_Blond_Hair_loss',
]].plot()
plt.xlabel('Epochs')
plt.ylabel('Validation Loss')
plt.legend(loc='upper left')
history_frame.loc[:, [
                  'val_Brown_Hair_loss',
                  'val_Eyeglasses_loss',
                  'val_Gray_Hair_loss',
                  'val_Heavy_Makeup_loss',
                  'val_High_Cheekbones_loss',
]].plot()
plt.xlabel('Epochs')
plt.ylabel('Validation Loss')
history_frame.loc[:, [
                  'val_Mouth_Slightly_Open_loss',
                  'val_Mustache_loss',
                  'val_Narrow_Eyes_loss',
                  'val_Rosy_Cheeks_loss',
                  'val_Smiling_loss',
]].plot()
plt.xlabel('Epochs')
plt.ylabel('Validation Loss')

history_frame.loc[:, [
                  'val_Straight_Hair_loss',
                  'val_Wavy_Hair_loss',
                  'val_Wearing_Earrings_loss',
                  'val_Wearing_Hat_loss',
                  'val_Wearing_Lipstick_loss',
                  'val_Wearing_Necklace_loss',
]].plot()

In [ ]:
val_accuracies = []
for col_name in history_frame.columns:
    if col_name.startswith('val_') and col_name.endswith('accuracy'):
        val_accuracies.append(history_frame[col_name])


In [ ]:
val_losses = []
for col_name in history_frame.columns:
    if col_name.startswith('val_') and col_name.endswith('loss'):
        val_losses.append(history_frame[col_name])

In [ ]:
import numpy as np
avg_val_losses = np.mean(val_losses, axis=0)
avg_val_accuracies = np.mean(val_accuracies, axis=0)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(avg_val_losses)
plt.title('Average Validation Loss. vs. Epochs (Set 6)')
plt.xlabel('Epochs')
plt.ylabel('Average Validation Loss')
plt.show()

plt.plot(avg_val_accuracies)
plt.title('Average Validation Acc. vs. Epochs (Set 6)')
plt.xlabel('Epochs')
plt.ylabel('Average Validation Accuracy')
plt.show()
